In [67]:
## Import packages

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math

In [68]:
## Load data from files

df1 = pd.read_csv('../hw2_data/D1.txt', sep=' ')
df2 = pd.read_csv('../hw2_data/D2.txt', sep=' ')
dfruns = pd.read_csv('../hw2_data/Druns.txt', sep=' ')

In [69]:
## Convert to numpy arrays for ease of use

d1 = df1.to_numpy()
d2 = df2.to_numpy()
druns = dfruns.to_numpy()

In [82]:
## Helper functions

# Generator for candidate splits
# Returns j, c where
# - j is the index of the split
# - c is the value at the split
def get_candidate_splits(data):
    splits = []
    
    # Split by first feature
    x1_set = set()
    for x1 in data[:, 0]:
        split = 0, x1
        if x1 not in x1_set:
            splits.append(split)
            x1_set.add(x1)
    
    # Split by second feature
    x2_set = set()
    for x2 in data[:, 1]:
        split = 1, x2
        if x2 not in x2_set:
            splits.append(split)
            x2_set.add(x2)
            
    return splits

# Majority label in the data
# Useful to determine the label of the leaf
# When no majority, predict 1
def get_majority_label(data):
    count_ones = np.count_nonzero(data[:, 2])
    self.label = 1 if 2 * count_ones >= len(data) else 0

def get_prob_entropy(prob_ones):
    if math.isclose(prob_ones, 0) or math.isclose(prob_ones, 1):
        return 0
    
    prob_zeros = 1 - prob_ones
    return -(prob_ones * math.log2(prob_ones) + prob_zeros * math.log2(prob_zeros))
    
def get_node_entropy(data):
    count_ones = np.count_nonzero(data[:, 2])
    total_count = len(data)
    prob_ones = count_ones / total_count
    return get_prob_entropy(prob_ones)

def get_data_split(data, j, c):
    then_data = data[data[:, j] >= c, :]
    else_data = data[data[:, j] < c, :]
    return then_data, else_data

In [87]:
## Tree data structure

class TreeNode:
    def __init__(data, then_child, else_child):
        self.then_child = then_child
        self.else_child = else_child
        if not then_child:
            # Leaf node. Find the label.
            self.label = get_majority_label(data)

In [88]:
def make_subtree(node_data):
    # Stopping criteria 1: node is empty
    if not node_data:
        return TreeNode(node_data, None, None)
    
    # Stopping criteria 2/3: all labels are same
    node_entropy = get_node_entropy(node_data)
    if math.isclose(node_entropy, 0):
        return TreeNode(node_data, None, None)
    
    # Find the best split
    max_gain_ratio = 0
    max_gain_split = None
    total_len = len(data)
    
    candidate_splits = get_candidate_splits(node_data)
    for j, c in candidate_splits:
        # Compute then and else splits
        then_split, else_split = get_data_split(node_data, j, c)
        
        # Find split entropy
        then_len = len(then_split)
        then_prob = then_len / total_len
        split_entropy = get_prob_entropy(then_prob)
        # Do not process 0 entropy splits
        if math.isclose(split_entropy, 0):
            continue
        
        # Find entropy of data after splitting
        then_entropy = get_node_entropy(then_split)
        else_entropy = get_node_entropy(else_split)
        post_split_entropy = then_prob * then_entropy + (1 - then_prob) * else_entropy
        
        # Compute gain ratio
        gain_ratio = (node_entropy - post_split_entropy) / split_entropy
            
        if max_gain_ratio < gain_ratio:
            max_gain_ratio = gain_ratio
            max_gain_split = then_split, else_split
            
    # Stoppping criteria 2/3: all non-trivial splits have zero gain ratio
    if not max_gain_ratio:
        return TreeNode(node_data, None, None)
    
    # Build the tree from children using the minimum split
    best_then_split, best_else_split = max_gain_split
    then_child, else_child = make_subtree(best_then_split), make_subtree(best_else_split)
    return TreeNode(node_data, then_child, else_child)